In [2]:
from nflows import models
import torch
from torchvision.transforms import ToPILImage

In [5]:
model = models.get_glow_model(2, (1,320,320))
model.load("/home/ubuntu/nflows/experiment_03_ocean/model_152.pth")

In [6]:
from collections import defaultdict
flow_data = defaultdict(dict)

def add_hook(name):
    def flow_hook(model, input, output):
        if not isinstance(input[0], int):
            flow_data[name]["input"] =  input[0].detach()
        else:
            flow_data[name]["input"] =  input[0]
        flow_data[name]["output"] = output[0].detach()
    return flow_hook

for fm_cnt, flow_module in enumerate(model.flows):
    for f_cnt, flow in enumerate(flow_module):
        flow.register_forward_hook(add_hook(f"flow_{fm_cnt}_{f_cnt}"))

# for q_cnt, q_module in enumerate(model.q0):
#     q_module.register_forward_hook(add_hook(f"q0_{q_cnt}"))
# with torch.no_grad():
s = model.sample(1)
flow_data

defaultdict(dict,
            {'q0_0': {'input': 1,
              'output': tensor([[[[ 0.4285,  0.6232,  1.6359,  ..., -1.4345,  1.5354,  0.7072],
                        [ 0.3388, -1.7125, -0.0979,  ...,  1.1370, -1.1651, -1.9183],
                        [ 0.4911,  1.6895,  0.5367,  ...,  0.9681, -0.1879,  0.4318],
                        ...,
                        [ 0.2823,  0.3053, -0.2043,  ...,  0.2118,  0.8639,  0.2712],
                        [-1.2097,  0.4526, -0.2934,  ...,  0.2861,  1.1482,  0.8035],
                        [ 0.2614, -1.7031, -1.8473,  ..., -2.3549,  0.0234, -0.4771]],
              
                       [[ 1.4486, -1.7367, -1.4307,  ..., -0.8695, -1.7418, -0.0812],
                        [ 0.2120,  0.2299,  0.6166,  ..., -0.1881,  0.2443, -0.6984],
                        [-0.4902,  0.3862, -0.7209,  ..., -0.3921,  0.2823, -1.7925],
                        ...,
                        [-0.3811, -0.2625, -1.4187,  ...,  1.9806,  0.4233,  0.8375],
    

In [7]:
transform = ToPILImage()
transform(s[0].squeeze(0))
transform(flow_data["output"].squeeze(0))

AttributeError: 'dict' object has no attribute 'squeeze'

In [8]:
from PIL import Image
def get_concat_h_blank(im1, im2, color=(0, 0, 0)):
    dst = Image.new('RGB', (im1.width + im2.width, max(im1.height, im2.height)), color)
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, 0))
    return dst

def get_concat_h_multi_blank(im_list):
    _im = im_list.pop(0)
    for im in im_list:
        _im = get_concat_h_blank(_im, im)
    return _im

def get_concat_v_multi_resize(im_list, resample=Image.Resampling.BICUBIC):
    min_width = min(im.width for im in im_list)
    im_list_resize = [im.resize((min_width, int(im.height * min_width / im.width)),resample=resample)
                      for im in im_list]
    total_height = sum(im.height for im in im_list_resize)
    dst = Image.new('RGB', (min_width, total_height))
    pos_y = 0
    for im in im_list_resize:
        dst.paste(im, (0, pos_y))
        pos_y += im.height
    return dst

In [9]:
images = []
transform = ToPILImage()
for k, v in flow_data.items():
    images.append(transform(v["input"].squeeze(0).mean(0)))

get_concat_h_multi_blank(images)

AttributeError: 'int' object has no attribute 'squeeze'

In [37]:
model

MultiscaleFlow(
  (q0): ModuleList(
    (0): ClassCondDiagGaussian()
    (1): ClassCondDiagGaussian()
    (2): ClassCondDiagGaussian()
  )
  (flows): ModuleList(
    (0): ModuleList(
      (0): GlowBlock(
        (flows): ModuleList(
          (0): AffineCouplingBlock(
            (flows): ModuleList(
              (0): Split()
              (1): AffineCoupling(
                (param_map): ConvNet2d(
                  (net): Sequential(
                    (0): Conv2d(8, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
                    (1): LeakyReLU(negative_slope=0.0)
                    (2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
                    (3): LeakyReLU(negative_slope=0.0)
                    (4): Conv2d(256, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
                  )
                )
              )
              (2): Merge()
            )
          )
          (1): Invertible1x1Conv()
          (2): ActNorm()
        )
      )
   

In [6]:
model.flows

MultiscaleFlow(
  (q0): ModuleList(
    (0): ClassCondDiagGaussian()
    (1): ClassCondDiagGaussian()
    (2): ClassCondDiagGaussian()
  )
  (flows): ModuleList(
    (0): ModuleList(
      (0): GlowBlock(
        (flows): ModuleList(
          (0): AffineCouplingBlock(
            (flows): ModuleList(
              (0): Split()
              (1): AffineCoupling(
                (param_map): ConvNet2d(
                  (net): Sequential(
                    (0): Conv2d(8, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
                    (1): LeakyReLU(negative_slope=0.0)
                    (2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
                    (3): LeakyReLU(negative_slope=0.0)
                    (4): Conv2d(256, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
                  )
                )
              )
              (2): Merge()
            )
          )
          (1): Invertible1x1Conv()
          (2): ActNorm()
        )
      )
   

In [45]:
model.q0[0].hoo

TypeError: 'ClassCondDiagGaussian' object is not subscriptable